# collect youtube ranking data and visualize only Selenium

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [14]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [15]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [21]:
# pop-up window exist, close pop-up and back to main window
widnow_len = len(driver.window_handles)
if widnow_len == 2:
  driver.switch_to.window(driver.window_handles[1])
  driver.close()

driver.switch_to.window(driver.window_handles[0])

# If you don't PAGE_DOWN, just show part of the page 
# so you have to PAGE_DOWN first
body = driver.find_element_by_css_selector('body')
for i in range(30):
  body.send_keys(Keys.PAGE_DOWN)
  time.sleep(1)

In [22]:
# soup.select('form > table > tbody > tr')
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [23]:
def covert_unit(s):
  
  s = s.replace('억', '').replace('개', '').replace(',', '')
  s = s.replace('만', '0000')
  return f'{int(s):,d}'

In [25]:
from tqdm.notebook import tqdm

In [27]:
results = []
for page in tqdm(range(1,11)):
  url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
  driver.get(url)
  time.sleep(3)

  widnow_len = len(driver.window_handles)
  if widnow_len == 2:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
  driver.switch_to.window(driver.window_handles[0])

  # If you don't PAGE_DOWN, just show part of the page 
  # so you have to PAGE_DOWN first
  body = driver.find_element_by_css_selector('body')
  for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

  trs = driver.find_elements_by_css_selector('.aos-init')

  for tr in trs:
    category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
  # category = channel.select_one('p.category').get_text().strip(' \n[] ')
    name = tr.find_element_by_css_selector('.subject a').text.strip()
  # name = channel.select_one('.subject a').text.strip()
    subscriber = covert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
  # subscriber = channel.select_one('.subscriber_cnt').text
    view = covert_unit(tr.find_element_by_css_selector('.view_cnt').text)
  # view = channel.select_one('.view_cnt').text
    video = covert_unit(tr.find_element_by_css_selector('.video_cnt').text)
  # video = channel.select_one('.video_cnt').text[:-1]
    results.append([category, name, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,100,000","19,018,090,000",371
1,음악/댄스/가수,HYBE LABELS,"60,400,000","18,775,490,000",655
2,음악/댄스/가수,BANGTANTV,"56,500,000","12,212,970,000","1,581"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,925,740,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


In [30]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,미분류,복지마블TV [Welfare Marble],"510,000","50,210,000",242
996,게임,OGN,"510,000","609,090,000","27,433"
997,게임,미소,"510,000","210,780,000","3,084"
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,두치와뿌꾸,"510,000","780,240,000","9,901"
